### This notebook is currently in append mode, starting from t:

This is because the write failed after 26, so I will rewrite 26-105, saving ~5 days in the process

Changes:

1. Skip creating new zarr store

2. Change timestep iteration in the last cell

3. Disabled the `mv prev_iter new_iter`

In [1]:
t_start = 0

In [2]:
%cd /home/idies/workspace/turbulence-ceph-staging/sciserver-turbulence

/home/idies/workspace/turbulence-ceph-staging/sciserver-turbulence


In [3]:
import os

NCAR_JHF_BASE_PATH = '/home/idies/workspace/turbulence-ceph-staging/ncar-jhf'

NCAR_JHF_HR_PATH = os.path.join(NCAR_JHF_BASE_PATH, 'hr')
NCAR_JHF_LR_PATH = os.path.join(NCAR_JHF_BASE_PATH, 'lr')

NCAR_FILES_EXTENSION = '.nc'

In [4]:
DATASET_TO_PROCESS = NCAR_JHF_HR_PATH

In [5]:
"""
import packages
"""
import os
import sys
import zarr
import itertools
import numpy as np
from tqdm.auto import tqdm
from zarr.storage import DirectoryStore
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from giverny.turbulence_dataset import *
from giverny.turbulence_toolkit import *
import xarray as xr
import traceback
import dask

In [6]:
"""
paths
"""
dataset_title = 'stsabl2048high'
store_path = f"/home/idies/workspace/turbulence-ceph-staging/sciserver-turbulence/{dataset_title}/{dataset_title}.zarr"
store = DirectoryStore(store_path, dimension_separator = '/')

In [7]:
"""
time offset from 0 when querying the dataset
"""
time_offset = 1

"""
# offset from the first time chunk to write data into the zarr store on ceph.
# if this value = 0, that means time 0 is written to the zarr time chunk folder = 0.
# if this value = 1, that means time 0 is written to the zarr time chunk folder = 1. this is needed to keep a placeholder time chunk folder for pchip interpolation
# because the precursor time cannot be read by giverny getCutout and will have to be read and written manually.
"""
time_ceph_offset = 0

In [8]:
# xyz and time dimensions for the full dataset including times for pchip interpolation that cannot be read by giverny.
xyzt_dims_full = [2048, 2048, 2048, 105]
# xyz and time dimensions for the dataset to be read with giverny.
xyzt_dims = [2048, 2048, 2048, 105]

In [9]:
"""
zarr store parameters
"""

# xyz and time chunk sizes for the zarr store on ceph.
xyzt_chunk_sizes = [64, 64, 64, 1]

# xyz and time dimensions to query in parallel when reading from the legacy stores.
# Ariel: This is also used to keep a constraint on memory usage for non-FileDB queries. Don't remove this, it's not incorrect
xyzt_filedb_file_dims = np.array([512, 512, 512, 1])

# use the default stride value of 1 for each axis when using giverny to retrieve a cutout of the data.
strides = [1, 1, 1]

# map the zarr group variables to the number of values stored for each grid point.
zarr_groups = {
    'velocity': 3,
    'pressure': 1,
    'temperature': 1,
    'sgsenergy': 1
}
zarr_variables = list(zarr_groups.keys())
num_variables = len(zarr_variables)

In [10]:
"""
runtime parameters
"""

# number of workers to read in parallel.
# Keep this under 4, this reads 50GB per worker per worker
num_workers = 5
# maximum number of retries in case of an error.
max_retries = 4
# output path for writing the report text file.
output_path = os.path.join('/home/idies/workspace/turbulence-ceph-staging/sciserver-turbulence/reports/', dataset_title)

### Make sure to disable this if just verifying data

In [11]:
# !mv /home/idies/workspace/turbulence-ceph-staging/sciserver-turbulence/stsabl2048high /home/idies/workspace/turbulence-ceph-staging/sciserver-turbulence/stsabl2048high_OLD9

In [12]:
# """
# create the zarr store
# """
# def create_zarr_store(store, xyzt_dims_full, xyzt_chunk_sizes, zarr_groups):
#     dt = np.dtype(np.float32)
#     dt = dt.newbyteorder('<')

#     # create the zarr store.
#     root = zarr.group(store = store, overwrite = True, synchronizer = None)

#     # create the zarr group for each variable.
#     for variable_name in zarr_groups:
#         zarr_dims = zarr_groups[variable_name]

#         zarr_group = root.zeros(variable_name, shape = (xyzt_dims_full[3], xyzt_dims_full[2], xyzt_dims_full[1], xyzt_dims_full[0], zarr_dims),
#                                 chunks = (xyzt_chunk_sizes[3], xyzt_chunk_sizes[2], xyzt_chunk_sizes[1], xyzt_chunk_sizes[0], zarr_dims),
#                                 dtype = dt, compressor = None)

#     print('zarr store created.')
#     print('-')
#     sys.stdout.flush()


# def confirm_overwrite():
#     while True:
#         response = input("Are you sure you want to overwrite everything? (y/n): ").strip().lower()
#         if response in ('y', 'n'):
#             return response == 'y'  # Returns True for 'y', False for 'n'
#         print("Invalid input. Please enter 'y' or 'n'.")

# # if confirm_overwrite():
# #     create_zarr_store(store, xyzt_dims_full, xyzt_chunk_sizes, zarr_groups)
# # else:
# #     print("Operation canceled.")

In [11]:
import dask.array as da

def merge_velocities(transposed_ds, chunk_size_base=64):
    """
        Merge the 3 velocity components/directions - such merging
        exhibits faster 3-component reads. This is a Dask lazy
         computation
    """

    # Merge Velocities into 1
    b = da.stack([transposed_ds['u'], transposed_ds['v'], transposed_ds['w']], axis=3)
    b = b.squeeze()  # It should be (2048, 2048, 2048, 3, 1) before this. Use (2048, 2048, 2048, 3)
    # Make into correct chunk sizes
    b = b.rechunk((chunk_size_base, chunk_size_base, chunk_size_base, 3))  # Dask chooses (64,64,64,1)
    result = transposed_ds.drop_vars(['u', 'v', 'w'])  # Drop individual velocities

    # Add joined velocity to original group
    # Can't make the dim name same as scalars
    result['velocity'] = xr.DataArray(b, dims=(
        'nnz', 'nny', 'nnx', 'velocity component (xyz)'))

    return result


import re

def select_file(folder_path, timestep, file_acronym='jhf'):
    file_list = os.listdir(folder_path)

    for filename in file_list:
        # Extract the number from the filename using a more specific regular expression
        match = re.search(file_acronym+r'\.(\d+)\.nc', filename)
        if match:
            file_timestep = int(match.group(1))
            if file_timestep == timestep:
                # Return full paths
                return os.path.join(folder_path, filename)
    # If no file is found, raise an exception
    raise FileNotFoundError(f"No file found for timestep {timestep}")

## Begin Write

In [12]:
"""
read JHTDB datasets from fileDB using giverny and then write to a zarr store on ceph
"""

# open the zarr store using DirectoryStore in read-only mode, to verify
root = zarr.open(store, mode = 'r')

# # append mode to edit
# root = zarr.open(store, mode = 'a')

def process_cube(coords):
    try:
        """
        Ariel changes to code here for reading in the new NCAR data without giverny.
        """

        """
        Select relevant 512-subcube from 2048^3 original data
        This reduces peak memory consumption
        """
        # Get current timestep no. coords = (x, y, z, time), where x,y,z are 512-chunk indices
        timestep = coords[3]
        # Unpack cube coordinates
        i, j, k = coords[:3]  # i: X-index, j: Y-index, k: Z-index

        # chunks=512^3 bcs we want to load a 512^3 subset, to keep memory usage low
        # Ariel: Not necessary to move the data_xr out of the function, and pass data_subcube as argument, as there's minimal overhead here
        data_xr = xr.open_dataset(select_file(DATASET_TO_PROCESS, timestep),
                                  chunks={'nnz': xyzt_filedb_file_dims[2],
                                          'nny': xyzt_filedb_file_dims[1],
                                          'nnx': xyzt_filedb_file_dims[0]})

        assert isinstance(data_xr['e'].data, dask.array.core.Array)

        # Define mapping of dimension name to chunk size
        # Currently Redundant since 512^3 dims are the same along all dimensions
        chunk_sizes_dict = {
            'nnx': xyzt_filedb_file_dims[0],
            'nny': xyzt_filedb_file_dims[1],
            'nnz': xyzt_filedb_file_dims[2],
        }

        # Find and load the correct 512^3 subcube using coords x-y-z
        selection = {
            'nnz': slice(k * chunk_sizes_dict['nnz'], (k + 1) * chunk_sizes_dict['nnz']),
            'nny': slice(j * chunk_sizes_dict['nny'], (j + 1) * chunk_sizes_dict['nny']),
            'nnx': slice(i * chunk_sizes_dict['nnx'], (i + 1) * chunk_sizes_dict['nnx']),
        }

        # 512-subcube to manage memory constraints
        data_subcube = data_xr.isel(selection)


        """
        Preprocess this subcube for writing to Zarr
        """

        # Group 3 velocity components together
        merged_velocity = merge_velocities(data_subcube, chunk_size_base=xyzt_chunk_sizes[0])

        # Add an extra dimension to the data to match isotropic8192
        for v in ['e','p','t']:
            merged_velocity[v] = merged_velocity[v].expand_dims({'extra_dim': [1]}).drop_vars('extra_dim')
            # Put the extra dimension in the back
            merged_velocity[v] = merged_velocity[v].transpose('nnz', 'nny', 'nnx', 'extra_dim')


        # Rename variables
        merged_velocity = merged_velocity.rename({'e': 'sgsenergy', 't': 'temperature', 'p': 'pressure'})

        dataset = merged_velocity


        """
        Load the subcube into main memory
        """
        variable_data = []
        for zarr_variable in zarr_variables:
            variable_data.append(dataset[zarr_variable].values)


        """
        End of code for reading in data.
        """

        x, y, z = [coord * xyzt_filedb_file_dims[index] + 1 for index, coord in enumerate(coords[:3])]
        time = timestep

        # def save_store(giverny_cube, variable_name):
        #     root[variable_name][time + time_ceph_offset,
        #                         z - 1 : z + xyzt_filedb_file_dims[2] - 1,
        #                         y - 1 : y + xyzt_filedb_file_dims[1] - 1,
        #                         x - 1 : x + xyzt_filedb_file_dims[0] - 1, :] = giverny_cube

        def verify_copy(giverny_cube, variable_name):
            ceph_cube = root[variable_name][time + time_ceph_offset,
                                            z - 1 : z + xyzt_filedb_file_dims[2] - 1,
                                            y - 1 : y + xyzt_filedb_file_dims[1] - 1,
                                            x - 1 : x + xyzt_filedb_file_dims[0] - 1, :]

            if np.all(giverny_cube == ceph_cube):
                return "valid copy"
            else:
                return "corrupt copy"

        # # Save the variables in parallel.
        # with ThreadPoolExecutor(num_variables) as p:
        #     list(p.map(save_store, variable_data, zarr_variables))


        # Verify that the copies are not corrupt.
        with ThreadPoolExecutor(num_variables) as p:
            verified = list(p.map(verify_copy, variable_data, zarr_variables))
        if any([message == "corrupt copy" for message in verified]) or len(verified) != num_variables:
            return f"error processing cube at {coords}: verification failed"
        else:
            return f"successfully processed cube at {coords}"


    except Exception as e:
        tb = traceback.format_exc()
        raise Exception(f"error processing cube at {coords}: {type(e).__name__}, {e}\n{tb}")

In [ ]:
"""
Call process_cubes on the collection of 512^3 subchunks
"""

# Commented out the suppress_stderr context manager
# @contextlib.contextmanager
# def suppress_stderr():
#     with open(os.devnull, 'w') as devnull:
#         with contextlib.redirect_stderr(devnull):
#             yield

# Write the report file.
if not os.path.exists(output_path):
    os.mkdir(output_path)

# Process cubes in parallel.
cube_coords = list(itertools.product(*[range(dim // chunk) for dim, chunk in zip(xyzt_dims[:3], xyzt_filedb_file_dims[:3])]))

current_time_pbar = tqdm(total=len(cube_coords), desc="chunks completed", leave=False)

with open(os.path.join(output_path, f"{dataset_title}_report-success.txt"), "w") as f_success:
    with open(os.path.join(output_path, f"{dataset_title}_report-error.txt"), "w") as f_error:
        for time in tqdm(range(t_start, xyzt_dims[-1]), # TODO Ariel changed to start from >0 timestep
                 total=xyzt_dims[-1] - t_start, desc="time"):
            correct_flag = False
            retries = 0
            while not correct_flag and retries <= max_retries:
                retries += 1

                # Update cube_coords with the time.
                cube_coords_time = [(x, y, z, time) for x, y, z in cube_coords]

                results = []
                try:
                    with ProcessPoolExecutor(num_workers) as executor:
                        # Removed suppress_stderr()
                        for result in executor.map(process_cube, cube_coords_time):
                            results.append(result)
                            current_time_pbar.update(1)

                            if "error processing cube" in result:
                                break

                    # Reset the chunk progress bar.
                    current_time_pbar.reset()

                    print('\r' + ' ' * 100 + '\r', end='', flush=True)
                    if any(["error processing cube" in message for message in results]) or results == []:
                        if retries > max_retries:
                            output_str = f"error consolidating data for time = {time}\n"
                            f_error.write(output_str)
                            f_error.flush()

                            print(f"\rerror consolidating data for time = {time}", end='')
                        else:
                            print(f"\rretrying (n = {retries}) time = {time}", end='')
                            continue
                    else:
                        output_str = f"successfully consolidated data for time = {time}\n"
                        f_success.write(output_str)
                        f_success.flush()

                        print(f"\rsuccessfully consolidated data for time = {time}", end='')

                        correct_flag = True
                except Exception as e:
                    tb = traceback.format_exc()
                    print('\r' + ' ' * 100 + '\r', end='', flush=True)

                    # Reset the chunk progress bar.
                    current_time_pbar.reset()

                    error_message = f"code exception when consolidating data for time = {time}: {type(e).__name__}, {e}\n{tb}\n"
                    f_error.write(error_message)
                    f_error.flush()

                    print(f"\rcode exception when consolidating data for time = {time}: {e}", end='')
                    if retries > max_retries:
                        break
                    else:
                        print(f"\rretrying (n = {retries}) time = {time}", end='')
                        continue

print('\n-')
print("completed zarr consolidation, check report files")

chunks completed:   0%|          | 0/64 [00:00<?, ?it/s]

time:   0%|          | 0/105 [00:00<?, ?it/s]